In [15]:
!pip install folium

  Using cached folium-0.18.0-py2.py3-none-any.whl (108 kB)
  Using cached branca-0.8.1-py3-none-any.whl (26 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl (134 kB)
  Attempting uninstall: jinja2
    Found existing installation: Jinja2 2.11.3
    Uninstalling Jinja2-2.11.3:
      Successfully uninstalled Jinja2-2.11.3


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
anaconda-project 0.9.1 requires ruamel-yaml, which is not installed.
sphinx 4.0.1 requires Jinja2<3.0,>=2.3, but you have jinja2 3.1.6 which is incompatible.
sphinx 4.0.1 requires MarkupSafe<2.0, but you have markupsafe 2.1.5 which is incompatible.

[notice] A new release of pip available: 22.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import requests
import time
import pandas as pd
import folium
from folium.plugins import MarkerCluster

In [4]:
API_KEY = 'nuh uh'  
HEADERS = {'Authorization': f'Bearer {API_KEY}'}
SEARCH_ENDPOINT = 'https://api.yelp.com/v3/businesses/search'

In [5]:
categories = ['chinese', 'japanese', 'korean', 'vietnamese', 'indian', 'mexican', 'thai', 'italian', 'pizza', 'burgers']
postal_codes = ['V6V', 'V6W', 'V6X', 'V6Y', 'V7A', 'V7C', 'V7E']

all_results = []

for category in categories:
    for postal in postal_codes:
        print(f"Searching: {category} in {postal}...")
        offset = 0
        while offset <= 190: 
            params = {
                'location': f'{postal}, Richmond, BC',
                'categories': f'{category},restaurants',
                'limit': 50,
                'offset': offset
            }

            response = requests.get(SEARCH_ENDPOINT, headers=HEADERS, params=params)
            if response.status_code != 200:
                print(f"Error {response.status_code}: {response.text}")
                break

            data = response.json()
            businesses = data.get('businesses', [])
            if not businesses:
                break

            for biz in businesses:
                all_results.append({
                    'id': biz['id'],
                    'name': biz['name'],
                    'address': ", ".join(biz['location']['display_address']),
                    'latitude': biz['coordinates']['latitude'],
                    'longitude': biz['coordinates']['longitude'],
                    'categories': ", ".join([cat['title'] for cat in biz.get('categories', [])])
                })

            offset += 50
            time.sleep(0.5)

# drop duplicates
df = pd.DataFrame(all_results)
df.drop_duplicates(subset='id', inplace=True)
df.reset_index(drop=True, inplace=True)
df

Searching: chinese in V6V...
Searching: chinese in V6W...
Searching: chinese in V6X...
Searching: chinese in V6Y...
Searching: chinese in V7A...
Searching: chinese in V7C...
Searching: chinese in V7E...
Searching: japanese in V6V...
Searching: japanese in V6W...
Searching: japanese in V6X...
Searching: japanese in V6Y...
Searching: japanese in V7A...
Searching: japanese in V7C...
Searching: japanese in V7E...
Searching: korean in V6V...
Searching: korean in V6W...
Searching: korean in V6X...
Searching: korean in V6Y...
Searching: korean in V7A...
Searching: korean in V7C...
Searching: korean in V7E...
Searching: vietnamese in V6V...
Searching: vietnamese in V6W...
Searching: vietnamese in V6X...
Searching: vietnamese in V6Y...
Searching: vietnamese in V7A...
Searching: vietnamese in V7C...
Searching: vietnamese in V7E...
Searching: indian in V6V...
Searching: indian in V6W...
Searching: indian in V6X...
Searching: indian in V6Y...
Searching: indian in V7A...
Searching: indian in V7C...

,id,name,address,latitude,longitude,categories
0,y7_33jucQRo37eQEshPeMQ,Shoestring Cafe,"4611 No 6 Road, Suite 180, Richmond, BC V6V 2L...",49.179874,-123.069558,"French, Seafood, Steakhouses"
1,pgyPhwk3lB1DxedDzygUUg,The Fish Man,"1170-8391 Alexandra Road, Richmond, BC V6X 1C3...",49.178466,-123.130252,"Seafood, Chinese, Barbeque"
2,Hr5sm6NpE6K-98PrqHksbQ,Anar Persian Cuisine,"12051 Third Avenue, Richmond, BC V7E 3A2, Canada",49.126127,-123.185905,"Persian/Iranian, Middle Eastern"
3,3H63O0o-PXy7yW3MtJznOQ,The Story Cafe,"2800 Sexsmith Road, Richmond, BC V6X 1V5, Canada",49.193593,-123.127918,"Bars, Pizza, Italian"
4,4EV_ZcQmjAmP3pmO-_nb2A,Miku,"200 Granville Street, Suite 70, Vancouver, BC ...",49.286913,-123.113049,"Japanese, Sushi Bars"
...,...,...,...,...,...,...
471,CHYGStOcv5MgdIwBpSDgSg,John's Pan Pizza,"12420 No 1 Road, Suite 130, Richmond, BC V7E 6...",49.124770,-123.180870,"Pizza, Salad, Italian"
472,p4bEiFv80KGqdpHztWaD_Q,Good Co Bar - Steveston,"4020 Bayview Street, Steveston, BC V7E 0B3, Ca...",49.123578,-123.180234,"Bars, Chicken Wings"
473,iONl1L8eLbLmZFENHae41Q,Shady Island Seafood Bar & Grill,"3800 Bayview Street, Richmond, BC V7E 6K7, Canada",49.123923,-123.183891,"Seafood, Bars"
474,vKIpb6S6C3Cc4GvDorBn2w,Moncton Pizzeria,"3680 Moncton Street, Richmond, BC V7E 3A4, Canada",49.125025,-123.183609,Pizza


In [6]:
# --- Manually append Domino's and Torake ---
params_list = [
    {'term': "Domino's Pizza", 'location': 'Richmond, BC'},
    {'term': "Torake", 'location': 'Richmond, BC'}
]

for params in params_list:
    params['limit'] = 1
    response = requests.get(SEARCH_ENDPOINT, headers=HEADERS, params=params)
    biz = response.json().get('businesses', [None])[0]
    if not biz or biz['id'] in df['id'].values:
        continue

    coords = biz['coordinates']
    df.loc[len(df)] = {
        'id': biz['id'],
        'name': biz['name'],
        'latitude': coords['latitude'],
        'longitude': coords['longitude'],
        'categories': ", ".join([cat['title'] for cat in biz.get('categories', [])]),
        'address': ", ".join(biz['location']['display_address']),
    }

df.reset_index(drop=True, inplace=True)
print(f"Final total restaurants: {len(df)}")
df

Final total restaurants: 478


,id,name,address,latitude,longitude,categories
0,y7_33jucQRo37eQEshPeMQ,Shoestring Cafe,"4611 No 6 Road, Suite 180, Richmond, BC V6V 2L...",49.179874,-123.069558,"French, Seafood, Steakhouses"
1,pgyPhwk3lB1DxedDzygUUg,The Fish Man,"1170-8391 Alexandra Road, Richmond, BC V6X 1C3...",49.178466,-123.130252,"Seafood, Chinese, Barbeque"
2,Hr5sm6NpE6K-98PrqHksbQ,Anar Persian Cuisine,"12051 Third Avenue, Richmond, BC V7E 3A2, Canada",49.126127,-123.185905,"Persian/Iranian, Middle Eastern"
3,3H63O0o-PXy7yW3MtJznOQ,The Story Cafe,"2800 Sexsmith Road, Richmond, BC V6X 1V5, Canada",49.193593,-123.127918,"Bars, Pizza, Italian"
4,4EV_ZcQmjAmP3pmO-_nb2A,Miku,"200 Granville Street, Suite 70, Vancouver, BC ...",49.286913,-123.113049,"Japanese, Sushi Bars"
...,...,...,...,...,...,...
473,iONl1L8eLbLmZFENHae41Q,Shady Island Seafood Bar & Grill,"3800 Bayview Street, Richmond, BC V7E 6K7, Canada",49.123923,-123.183891,"Seafood, Bars"
474,vKIpb6S6C3Cc4GvDorBn2w,Moncton Pizzeria,"3680 Moncton Street, Richmond, BC V7E 3A4, Canada",49.125025,-123.183609,Pizza
475,W3fHTFr9TE6OwP2AOTRdhA,Kuroki Kaiseki Omakase Kitchen,"110-3880 Chatham Street, Richmond, BC V7E 2Z5,...",49.126214,-123.182128,"Japanese, Sushi Bars"
476,v6GRZxd-96gojJrXKFcLdw,Domino's Pizza,"9471 No. 2 Road, Unit 120, Richmond, BC V7E 2C...",49.144227,-123.159686,"Pizza, Chicken Wings, Sandwiches"


In [7]:
# --- Create Folium Map ---
m = folium.Map(location=[49.1666, -123.1336], zoom_start=13, tiles='CartoDB dark_matter')

# Add all restaurants
for _, row in df.iterrows():
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=1.5,
        color='#00ffff',
        fill=True,
        fill_opacity=1,
        stroke=False
    ).add_to(m)

# Save the map
m.save("richmond_restaurants_map.html")
print("🌐 Map saved as richmond_restaurants_map.html")

🌐 Map saved as richmond_restaurants_map.html
